In [3]:
# System libraries:
%load_ext autoreload
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
def import_yle_ru():
    # Original columns:
    #alternativeId - drop
    #authors - keep (roughly half of them are nulls, but non-nulls might be interesting)
    #dateJsonModified - drop
    #dateContentModified - drop
    #datePublished - keep
    #headline - keep and clean
    #id - keep
    #language - drop - they are all unique
    #lead - drop
    #lifespan - drop
    #mainMedia - drop - it's pictures
    #meta - drop - of the form {'costCenters': ['1423-novosti']}
    #properties - drop - of the form [importance:normal, uutisvahti:enabled]
    #publisher - drop - array(['Yle Novosti'], dtype=object)
    #sources - keep!
    #subjects - drop for now
    #url - keep
    #content0 - keep
    yle_ru_raw = None
    with open("data/yle_russian_articles.json",  encoding='utf-8') as json_data:
        yle_ru_raw = json_data.read().replace('}\n{', '},\n{')

    yle_ru_dict = json.loads("[" + yle_ru_raw + "]")

    yle_ru_data = pd.DataFrame.from_dict(yle_ru_dict)

    drop_list = ['alternativeId', 'dateJsonModified', 'dateContentModified', 'language', 'lead', 
                 'lifespan', 'mainMedia', 'meta', 'properties', 'publisher', 'subjects']

    yle_ru_data = yle_ru_data.drop(columns = drop_list)

    #Filter article texts so that the headings are not included in the "content" column, and the rest of the text is joined
    yle_ru_data['content'] = yle_ru_data['content'].apply(lambda s: ' '.join([a['text'] if a['type'] == 'text' else '' for a in s]))

    #Filter article headlines to keep only the russian version
    yle_ru_data['headline'] = yle_ru_data['headline'].apply(lambda s: s['full'])
    
    return yle_ru_data

data = import_yle_ru()
#data.to_csv('/var/www/html/work/compressed/data/russia_finland/processed/yle_ru_processed.csv', index=False)

lemmas = pd.read_csv('/var/www/html/work/compressed/data/russia_finland/processed/yle_ru_lemmas.csv')
data = pd.concat([data, lemmas], axis = 1)


In [ ]:
mod = False
yle_ru_data = None
if mod:
    print("Regenerating data")
    data = import_yle_ru()
else:
    print("Loading from CSV")
    data = pd.read_csv('/var/www/html/work/compressed/data/russia_finland/processed/yle_ru_processed.csv')
    
data.iloc[:5]

In [ ]:
# Lemmatization functions:
stop_words = stopwords.words('russian') + list(string.punctuation)
m = Mystem()

def tokenize_text(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return filtered_sentence
    
def lemmatize_text(text):
    return [m.lemmatize(w)[0] for w in text]

def tokenize_and_lemmatize(text):
    tokenized = tokenize_text(text)
    lemmatized = lemmatize_text(tokenized)
    return lemmatized

In [ ]:
# lemmatize content:
lemmas_content = data.content.apply(tokenize_and_lemmatize)
lemmas_content.name = 'lemmas_content'

# lemmatize headlines:
lemmas_headline = data.headline.apply(tokenize_and_lemmatize)
lemmas_headline.name = 'lemmas_headline'

In [92]:
lemmas = pd.concat([lemmas_headline, lemmas_content], axis = 1)
lemmas.to_csv('/var/www/html/work/compressed/data/russia_finland/processed/yle_ru_lemmas.csv', header = True, index = False)
#data = pd.concat([data, lemmas], axis = 1)

In [94]:
data.iloc[:5]

,authors,content,coverage,datePublished,headline,id,sources,url,lemmas_headline,lemmas_content
0,NaN,Министр по вопросам окружающей среды Вилле Ни...,local,2013-04-07T20:12:33+0300,Министр окружающей среды Ниинистё хочет переве...,3-6594601,NaN,"{'short': 'http://yle.fi/uutiset/6594601', 'fu...","[министр, окружающий, среда, ниинист, хотеть, ...","[министр, вопрос, окружающий, среда, вилла, ни..."
1,NaN,Труд финских работников занимает 8-е по дорог...,local,2013-04-07T20:50:16+0300,"Финский труд дешевле, чем труд в Германии",3-6594603,NaN,"{'short': 'http://yle.fi/uutiset/6594603', 'fu...","[финский, труд, дешево, труд, германия]","[труд, финский, работник, занимать, 8, дорогов..."
2,NaN,На дорогах Южной Финляндии летние ограничения...,local,2013-04-08T08:38:40+0300,Начинается постепенный переход на летнее огран...,3-6594605,NaN,"{'short': 'http://yle.fi/uutiset/6594605', 'fu...","[начинаться, постепенный, переход, летний, огр...","[на, дорога, южный, финляндия, летний, огранич..."
3,NaN,В Хельсинкском надворном суде начинается расс...,local,2013-04-08T08:57:17+0300,В надворном суде начинается разбирательство по...,3-6594611,NaN,"{'short': 'http://yle.fi/uutiset/6594611', 'fu...","[в, надворный, суд, начинаться, разбирательств...","[в, хельсинкский, надворный, суд, начинаться, ..."
4,NaN,Несмотря на все усилия и различные кампании п...,local,2013-04-08T09:18:42+0300,Летних рабочих мест не хватает для всех,3-6594606,NaN,"{'short': 'http://yle.fi/uutiset/6594606', 'fu...","[летний, рабочий, место, хватать]","[несмотря, усилие, различный, кампания, создан..."


In [276]:
# Target 1 - get a word cloud using TF-IDF for the whole dataset

corpus = data.lemmas_content.apply(lambda s: ' '.join(s))

def tf_idf_score(corpus):
    tf = TfidfVectorizer(min_df = 0, stop_words = None)
    tfidf_matrix =  tf.fit_transform(corpus)
    feature_names = tf.get_feature_names() 

    for i, row in enumerate(tfidf_matrix):
        idx = np.argsort(np.array(row))
        top_words.append(np.array(feature_names)[idx][-20:])

    return top_words
    
top_words = tf_idf_score(corpus)

data['tfidf_top_words'] = pd.Series(top_words)

[array(['зеленый', 'собственный', 'обсуждаться', 'правительственный',
        'вилла', 'переводить', 'рыба', 'охрана', 'научный', 'соблюдать',
        'популяция', 'рыболовство', 'семга', 'игнорировать', 'вопрос',
        'лесной', 'сельский', 'хозяйство', 'министерство', 'ниинист'],
       dtype='<U24'),
 array(['расход', 'средний', 'шведский', 'статистический', 'ниже',
        'дорогой', 'германия', 'дешево', 'евро', 'взнос', 'дороговизна',
        'вопреки', 'общепринятый', 'почасовой', 'косвенный', 'немецкий',
        'финский', 'оплата', 'рабочий', 'труд'], dtype='<U24'),
 array(['сатакунт', 'автомагистраль', 'постепенно', '120', 'меняться',
        'откладывать', 'первоначальный', 'скоростной', 'эстерботний',
        'южный', 'вторник', 'действовать', 'вступать', 'неделя', 'на',
        'км', 'дорога', 'скорость', 'летний', 'ограничение'], dtype='<U24'),
 array(['000', 'неположенный', 'причинение', 'выливать', 'для',
        'приговаривать', '17', 'исключительно', 'двое', 'надвор

In [291]:
# Target 1 - continued - get a word cloud using TF-IDF for the whole dataset

# Concatenate all lists in the data['tfidf_top_words'] and generate a histogram

from collections import Counter

tf_idf_flattened = [b for a in data['tfidf_top_words'] for b in a]

counts = Counter(tf_idf_flattened)

import operator
sorted_terms = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
#sorted_terms

from wordcloud import WordCloud
# Target 2 - get a word cloud using TF-IDF for each year



# Target 3 - get a word cloud using TF-IDF for custom date range

ImportError: No module named 'wordcloud'

['ямиярви',
 'ямсякоски',
 'ян',
 'январь',
 'янез',
 'яни',
 'янна',
 'янша',
 'япония',
 'японский',
 'яркий',
 'ярмо',
 'ярослав',
 'ярь',
 'ясный',
 'яухиайнен',
 'яхт',
 'яхтенный',
 'яянекоски',
 'яятинный']

In [280]:
data.iloc[:5]

,authors,content,coverage,datePublished,headline,id,sources,url,lemmas_headline,lemmas_content,tfidf_top_words
0,NaN,Министр по вопросам окружающей среды Вилле Ни...,local,2013-04-07T20:12:33+0300,Министр окружающей среды Ниинистё хочет переве...,3-6594601,NaN,"{'short': 'http://yle.fi/uutiset/6594601', 'fu...","[министр, окружающий, среда, ниинист, хотеть, ...","[министр, вопрос, окружающий, среда, вилла, ни...","[зеленый, собственный, обсуждаться, правительс..."
1,NaN,Труд финских работников занимает 8-е по дорог...,local,2013-04-07T20:50:16+0300,"Финский труд дешевле, чем труд в Германии",3-6594603,NaN,"{'short': 'http://yle.fi/uutiset/6594603', 'fu...","[финский, труд, дешево, труд, германия]","[труд, финский, работник, занимать, 8, дорогов...","[расход, средний, шведский, статистический, ни..."
2,NaN,На дорогах Южной Финляндии летние ограничения...,local,2013-04-08T08:38:40+0300,Начинается постепенный переход на летнее огран...,3-6594605,NaN,"{'short': 'http://yle.fi/uutiset/6594605', 'fu...","[начинаться, постепенный, переход, летний, огр...","[на, дорога, южный, финляндия, летний, огранич...","[сатакунт, автомагистраль, постепенно, 120, ме..."
3,NaN,В Хельсинкском надворном суде начинается расс...,local,2013-04-08T08:57:17+0300,В надворном суде начинается разбирательство по...,3-6594611,NaN,"{'short': 'http://yle.fi/uutiset/6594611', 'fu...","[в, надворный, суд, начинаться, разбирательств...","[в, хельсинкский, надворный, суд, начинаться, ...","[000, неположенный, причинение, выливать, для,..."
4,NaN,Несмотря на все усилия и различные кампании п...,local,2013-04-08T09:18:42+0300,Летних рабочих мест не хватает для всех,3-6594606,NaN,"{'short': 'http://yle.fi/uutiset/6594606', 'fu...","[летний, рабочий, место, хватать]","[несмотря, усилие, различный, кампания, создан...","[30, молодой, лето, год, профсоюз, различный, ..."


In [277]:
len(top_words)

28428

In [271]:
type(tfidf_matrix)

scipy.sparse.csr.csr_matrix

In [250]:
np.array(feature_names)[top_words_idx[0]]

IndexError: arrays used as indices must be of integer (or boolean) type

In [238]:
feature_names[True, False, True:]

TypeError: list indices must be integers or slices, not tuple

In [229]:
top_words

array([[   0., 1678., 1679., ...,  931., 1028., 1199.],
       [   0., 1679., 1680., ..., 1311., 1795., 2266.],
       [   0., 1675., 1676., ...,  604.,  933., 1273.],
       ...,
       [   0., 1673., 1674., ..., 1802., 2129.,  243.],
       [   0., 1673., 1674., ...,  315., 1739.,  155.],
       [   0., 1672., 1673., ...,  251.,  749., 2135.]])

In [226]:
top_words.shape
np.array(feature_names).shape
top_20_words_matrix = np.array(feature_names)[top_words[:,-20:]]

(100, 2535)

(2535,)

IndexError: arrays used as indices must be of integer (or boolean) type

In [213]:
corpus[99]

'в город вантаа остро стоять проблема безработица среди иммигрант среди женщина число безработный 4 раз высоко среди иммигрант –\n 2 раз высоко среди коренной население в 2010 год город вантаа уровень безработица среди иммигрант составлять 20 время среди коренной население аналогичный цифра 7 среди город столичный регион ситуация вантаа –\n самый тяжелый в хельсинки уровень безработица среди иммигрант составлять 18 эспоо –\n 13 иммигрант находиться неравный положение зависимость страна происхождение плохо дело обстоять выходец африка ближний восток среди женщина работа иметь 55 среди мужчина –\n 33 уровень безработица среди выходец страна балтия составлять около 10 часто иммигрант трудоустраивать сфера клининг строительство торговля'

In [144]:
data.iloc[0].content

' Министр по вопросам окружающей среды Вилле Ниинистё (Зеленые) критикует политику министерства сельского и лесного хозяйства. По мнению Ниинистё, министерство не во всем соблюдает правительственную программу. Ниинистё требует перевести вопросы рыболовства из министерства сельского и лесного хозяйство в собственное ведомство. По словам Ниинистё, давшего интервью Новостям ЮЛЕ, чиновники министерства игнорируют научную информацию о защите популяции рыб. В последнее время в финских СМИ обсуждается, в частности, вопрос об охране семги.'

In [155]:
data.iloc[:100].lemmas_content

0     [министр, вопрос, окружающий, среда, вилла, ни...
1     [труд, финский, работник, занимать, 8, дорогов...
2     [на, дорога, южный, финляндия, летний, огранич...
3     [в, хельсинкский, надворный, суд, начинаться, ...
4     [несмотря, усилие, различный, кампания, создан...
5     [на, первый, всемирный, цыганский, конгресс, л...
6     [в, февраль, резко, сокращаться, объем, импорт...
7     [на, рудник, талвиваар, обнаруживать, новый, у...
8     [лесопромышленный, концерн, упм, закрывать, бу...
9     [хельсинки, предлагать, водитель, автобус, воз...
10    [учитель, толкать, выгонять, шуметь, столовая,...
11    [по, слово, катайнен, маргарет, тэтчер, визион...
12    [программа, Yle, TV1, мот, сегодня, вечер, пер...
13    [в, хельсинки, понедельник, открываться, предс...
14    [правительство, должно, составлять, государств...
15    [течь, рудник, талвиваар, кайнуу, удаваться, п...
16    [проект, строительство, целлюлозно-бумажный, к...
17    [в, финляндия, выделяться, слишком, мало, 